## <font color='blue'>Merging yearly data from Watergap study</font>

**Author: Pedro Alonso<br>
v1: 05/06/2021**

# 1. Loading and Cleaning Data

In [61]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

Defining the `netCDF4_to_csv` function written by Mitch/Thea, which does the conversion to CSVs, which are easily manageable through Pandas.

In [2]:
def netCDF4_to_csv(nc4_key, nc4_url, yearN=1980): #Condition- function starts from year 1980, nc4_file_path -> str ,nc4_key -> str, datatype either monthly or yearly
    try:
        import requests
        import h5py
        import netCDF4
        from netCDF4 import Dataset
        import pandas as pd
        import numpy as np
        import datetime as dateformat
    except:
        print("Error - Required Libraries not installed")
        return
    
    try:
        r = requests.get(nc4_url, allow_redirects=True)
        open(nc4_key+'.nc4', 'wb').write(r.content)
        ds = Dataset(nc4_key+'.nc4')
    except:
        print('Error - Dataset not able to find the file')
        return
    
    
    if type(nc4_key) is not str:
        print('Error - Check input for key type')
        return
        
    if type(yearN) is not int:
        print('Error - Check input for year')
        return
    elif yearN < 0:
        print("Error- enter a positive number for year")
        return
        
    
    ### This will determine the number of years/months in the NC4 data   
    f = h5py.File(nc4_key+'.nc4', 'r')
    print(f.keys())
    size = f[nc4_key].shape
    num = size[0]
    
    key = ds[nc4_key][:]

    lat = ds['lat'][:]
    lon = ds['lon'][:]
    
    year = yearN
    count = 0

    # monthly data
    if num == 1392:
        start = (year - 1900 - 1)*12;
        datatype = 'Monthly_Output'
        print(datatype)
        
        #Jan of YearN to Dec of 2016
        for i in range(start,num):
            data = key[i,:,:]
            if i == start:
                month = 1
                data_pd = pd.DataFrame(data = data)
                lat = ds['lat'][:]
                data_pd.index =  lat
                lon = ds['lon'][:]
                data_pd.columns = lon
                data_pd = pd.melt(data_pd, ignore_index = False, var_name='lon', value_name=str(dateformat.datetime(year, 1, month).strftime('%m/%d/%Y')))               
            else:
                data_I = pd.DataFrame(data = data)
                data_I.index =  lat
                
                if i % 12 == 0:
                    year = year+1
                    month = 1
                else:
                    month = month + 1
                    
                data_I = pd.melt(data_I,ignore_index = False, value_name= str(dateformat.datetime(year, month,1).strftime('%m/%d/%Y')))
                data_pd[str(dateformat.datetime(year,month,1).strftime('%m/%d/%Y'))] = data_I.loc[:,str(dateformat.datetime(year,month,1).strftime('%m/%d/%Y'))]
        
        # Convert to Yearly Data
        data_ion = data_pd['lon']
        data_no_ion = data_pd.drop(columns=['lon'])
        new = (data_no_ion.groupby(pd.PeriodIndex(data_no_ion.columns, freq='Y'), axis=1).mean().rename(columns=lambda c: str(c).lower()))
        data_pd=pd.concat([data_ion, new], axis=1) 
        
    # yearly data
    elif num == 116:
        start = year - 1900 - 1;
        datatype = 'Yearly_Output'
        print(datatype)
        
        #YearN to 2016
        for i in range(start,num):
            data = key[i,:,:]
            if i == start:
                data_pd = pd.DataFrame(data = data)
                lat = ds['lat'][:]
                data_pd.index =  lat
                lon = ds['lon'][:]
                data_pd.columns = lon
                data_pd = pd.melt(data_pd, ignore_index = False, var_name='lon', value_name=str(yearN))
            else:
                data_I = pd.DataFrame(data = data)
                data_I.index =  lat
                year = year+1
                data_I = pd.melt(data_I,ignore_index = False, value_name= str(year))
                data_pd[str(year)] = data_I.loc[:,str(year)]
            
    data_pd.to_csv("watergap_"+nc4_key+"_"+datatype+".csv")
    return data_pd

In the Watergap database (https://doi.pangaea.de/10.1594/PANGAEA.918447?format=html#download), 5 of the variables are presented yearly. The rest is originally in a monthly format. This notebook **will worry about the yearly ones for the time being**.

The cell below converts the data of the 5 files to CSVs that go straight to our working directory, to be loaded immediately after.

In [3]:
netCDF4_to_csv(nc4_key='pdomuse', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_pdomuse_yearly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='pdomww', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_pdomww_yearly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='pinduse', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_pinduse_yearly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='pindww', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_pindww_yearly_1901_2016.nc4')
netCDF4_to_csv(nc4_key='plivuse', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_plivuse_yearly_1901_2016.nc4')

<KeysViewHDF5 ['lat', 'lon', 'pdomuse', 'time']>
Yearly_Output
<KeysViewHDF5 ['lat', 'lon', 'pdomww', 'time']>
Yearly_Output
<KeysViewHDF5 ['lat', 'lon', 'pinduse', 'time']>
Yearly_Output
<KeysViewHDF5 ['lat', 'lon', 'pindww', 'time']>
Yearly_Output
<KeysViewHDF5 ['lat', 'lon', 'plivuse', 'time']>
Yearly_Output


,lon,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
89.75,-179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89.25,-179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88.75,-179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88.25,-179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87.75,-179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-87.75,179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-88.25,179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-88.75,179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-89.25,179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df_pdomuse = pd.read_csv(r'watergap_pdomuse_Yearly_Output.csv')
df_pdomuse.head(5)               

,Unnamed: 0,lon,1980,1981,1982,1983,1984,1985,1986,1987,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,89.75,-179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,89.25,-179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,88.75,-179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,88.25,-179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,87.75,-179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**df_pdomuse basic info:** <br>
* 259,200 entries, 39 columns (2 for position, 37 for years between 1980 and 2016)
* latitutde seems to be unproperly named as "Unnamed: 0"
* latitude and longitude are the only two columns without missing values
* All the year columns have exactly 67,420 valid entries (~26%). The rest are possibly just lat/long positions not in the watergap study.

In [46]:
df_pdomuse.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259200 entries, 0 to 259199
Data columns (total 39 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  259200 non-null  float64
 1   lon         259200 non-null  float64
 2   1980        67420 non-null   float64
 3   1981        67420 non-null   float64
 4   1982        67420 non-null   float64
 5   1983        67420 non-null   float64
 6   1984        67420 non-null   float64
 7   1985        67420 non-null   float64
 8   1986        67420 non-null   float64
 9   1987        67420 non-null   float64
 10  1988        67420 non-null   float64
 11  1989        67420 non-null   float64
 12  1990        67420 non-null   float64
 13  1991        67420 non-null   float64
 14  1992        67420 non-null   float64
 15  1993        67420 non-null   float64
 16  1994        67420 non-null   float64
 17  1995        67420 non-null   float64
 18  1996        67420 non-null   float64
 19  19

<ipython-input-46-55710c823dc0>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_pdomuse.info(verbose=True, null_counts=True)


**Fixing the "lat" column:**

In [47]:
df_pdomuse.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_pdomuse.head(1)

,lat,lon,1980,1981,1982,1983,1984,1985,1986,1987,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,89.75,-179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Loading the other 4 CSVs:**

In [48]:
df_pdomww = pd.read_csv(r'watergap_pdomww_Yearly_Output.csv')
df_pdomuse.info(verbose=True, null_counts=True) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259200 entries, 0 to 259199
Data columns (total 39 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   lat     259200 non-null  float64
 1   lon     259200 non-null  float64
 2   1980    67420 non-null   float64
 3   1981    67420 non-null   float64
 4   1982    67420 non-null   float64
 5   1983    67420 non-null   float64
 6   1984    67420 non-null   float64
 7   1985    67420 non-null   float64
 8   1986    67420 non-null   float64
 9   1987    67420 non-null   float64
 10  1988    67420 non-null   float64
 11  1989    67420 non-null   float64
 12  1990    67420 non-null   float64
 13  1991    67420 non-null   float64
 14  1992    67420 non-null   float64
 15  1993    67420 non-null   float64
 16  1994    67420 non-null   float64
 17  1995    67420 non-null   float64
 18  1996    67420 non-null   float64
 19  1997    67420 non-null   float64
 20  1998    67420 non-null   float64
 21  1999    67

<ipython-input-48-798db6f151fb>:2: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_pdomuse.info(verbose=True, null_counts=True)


In [49]:
df_pinduse = pd.read_csv(r'watergap_pinduse_Yearly_Output.csv')
df_pinduse.info(verbose=True, null_counts=True) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259200 entries, 0 to 259199
Data columns (total 39 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  259200 non-null  float64
 1   lon         259200 non-null  float64
 2   1980        67420 non-null   float64
 3   1981        67420 non-null   float64
 4   1982        67420 non-null   float64
 5   1983        67420 non-null   float64
 6   1984        67420 non-null   float64
 7   1985        67420 non-null   float64
 8   1986        67420 non-null   float64
 9   1987        67420 non-null   float64
 10  1988        67420 non-null   float64
 11  1989        67420 non-null   float64
 12  1990        67420 non-null   float64
 13  1991        67420 non-null   float64
 14  1992        67420 non-null   float64
 15  1993        67420 non-null   float64
 16  1994        67420 non-null   float64
 17  1995        67420 non-null   float64
 18  1996        67420 non-null   float64
 19  19

<ipython-input-49-33117d675c2f>:2: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_pinduse.info(verbose=True, null_counts=True)


In [50]:
df_pindww = pd.read_csv(r'watergap_pindww_Yearly_Output.csv')
df_pindww.info(verbose=True, null_counts=True) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259200 entries, 0 to 259199
Data columns (total 39 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  259200 non-null  float64
 1   lon         259200 non-null  float64
 2   1980        67420 non-null   float64
 3   1981        67420 non-null   float64
 4   1982        67420 non-null   float64
 5   1983        67420 non-null   float64
 6   1984        67420 non-null   float64
 7   1985        67420 non-null   float64
 8   1986        67420 non-null   float64
 9   1987        67420 non-null   float64
 10  1988        67420 non-null   float64
 11  1989        67420 non-null   float64
 12  1990        67420 non-null   float64
 13  1991        67420 non-null   float64
 14  1992        67420 non-null   float64
 15  1993        67420 non-null   float64
 16  1994        67420 non-null   float64
 17  1995        67420 non-null   float64
 18  1996        67420 non-null   float64
 19  19

<ipython-input-50-c9a1c7b49e6c>:2: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_pindww.info(verbose=True, null_counts=True)


In [51]:
df_plivuse = pd.read_csv(r'watergap_pdomww_Yearly_Output.csv')
df_plivuse.info(verbose=True, null_counts=True) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259200 entries, 0 to 259199
Data columns (total 39 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  259200 non-null  float64
 1   lon         259200 non-null  float64
 2   1980        67420 non-null   float64
 3   1981        67420 non-null   float64
 4   1982        67420 non-null   float64
 5   1983        67420 non-null   float64
 6   1984        67420 non-null   float64
 7   1985        67420 non-null   float64
 8   1986        67420 non-null   float64
 9   1987        67420 non-null   float64
 10  1988        67420 non-null   float64
 11  1989        67420 non-null   float64
 12  1990        67420 non-null   float64
 13  1991        67420 non-null   float64
 14  1992        67420 non-null   float64
 15  1993        67420 non-null   float64
 16  1994        67420 non-null   float64
 17  1995        67420 non-null   float64
 18  1996        67420 non-null   float64
 19  19

<ipython-input-51-c7c3eb217fab>:2: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_plivuse.info(verbose=True, null_counts=True)


**Fixing first column naming for all:**

In [52]:
df_pdomww.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_pinduse.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_pindww.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)
df_plivuse.rename(columns={'Unnamed: 0' : 'lat'}, inplace=True)

***

# 2. Feature Engineering pre-Merge

In our first attempt, we have two options as how to merge the CSVs:<br>
* **Option 1: merging horizontally, with fixed lat + lon and 37 years for each variable (2 + 5 x 37 = 187 columns)**
* Option 2: merging with lists in each year's cells. We'd keep the 39 column format, but for each year's cell, we'd see a list of variables values. This option seems to have 2 disadvantages: data processing might become harder/incompatible with several solutions and it also gets quite ugly when instead of 5 variables, we have dozens of them.

By moving forward with `Option 1`, the first step is to add the variable's name to each of the years so we can identify each of them when we see the final dataframe.

In [53]:
df_pdomuse = df_pdomuse.add_suffix('_pdomuse')
df_pdomuse.head(3)

,lat_pdomuse,lon_pdomuse,1980_pdomuse,1981_pdomuse,1982_pdomuse,1983_pdomuse,1984_pdomuse,1985_pdomuse,1986_pdomuse,1987_pdomuse,...,2007_pdomuse,2008_pdomuse,2009_pdomuse,2010_pdomuse,2011_pdomuse,2012_pdomuse,2013_pdomuse,2014_pdomuse,2015_pdomuse,2016_pdomuse
0,89.75,-179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,89.25,-179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,88.75,-179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
df_pdomww = df_pdomww.add_suffix('_pdomww')
df_pinduse = df_pinduse.add_suffix('_pinduse')
df_pindww = df_pindww.add_suffix('_pindww')
df_plivuse = df_plivuse.add_suffix('_plivuse')

In [57]:
df_yearly_concat = pd.concat([df_pdomuse,
                              df_pdomww,
                              df_pinduse,
                              df_pindww,
                              df_plivuse],
                             axis=1)

In [58]:
df_yearly_concat.shape

(259200, 195)

**We must still drop the lat and lon columns that are the same for all 5 variables before getting the 187 expected columns.**

In [64]:
to_drop = ['lat_pdomww', 'lon_pdomww',
           'lat_pinduse', 'lon_pinduse',
           'lat_pindww', 'lon_pindww',
           'lat_plivuse', 'lon_plivuse']  

df_yearly_concat.drop(to_drop, inplace=True, axis=1)

In [65]:
df_yearly_concat.shape

(259200, 187)

Taking a look at the result:

In [67]:
df_yearly_concat.head(5)

,lat_pdomuse,lon_pdomuse,1980_pdomuse,1981_pdomuse,1982_pdomuse,1983_pdomuse,1984_pdomuse,1985_pdomuse,1986_pdomuse,1987_pdomuse,1988_pdomuse,1989_pdomuse,1990_pdomuse,1991_pdomuse,1992_pdomuse,1993_pdomuse,1994_pdomuse,1995_pdomuse,1996_pdomuse,1997_pdomuse,1998_pdomuse,1999_pdomuse,2000_pdomuse,2001_pdomuse,2002_pdomuse,2003_pdomuse,2004_pdomuse,2005_pdomuse,2006_pdomuse,2007_pdomuse,2008_pdomuse,2009_pdomuse,2010_pdomuse,2011_pdomuse,2012_pdomuse,2013_pdomuse,2014_pdomuse,2015_pdomuse,2016_pdomuse,1980_pdomww,1981_pdomww,1982_pdomww,1983_pdomww,1984_pdomww,1985_pdomww,1986_pdomww,1987_pdomww,1988_pdomww,1989_pdomww,1990_pdomww,1991_pdomww,1992_pdomww,1993_pdomww,1994_pdomww,1995_pdomww,1996_pdomww,1997_pdomww,1998_pdomww,1999_pdomww,2000_pdomww,2001_pdomww,2002_pdomww,2003_pdomww,2004_pdomww,2005_pdomww,2006_pdomww,2007_pdomww,2008_pdomww,2009_pdomww,2010_pdomww,2011_pdomww,2012_pdomww,2013_pdomww,2014_pdomww,2015_pdomww,2016_pdomww,1980_pinduse,1981_pinduse,1982_pinduse,1983_pinduse,1984_pinduse,1985_pinduse,1986_pinduse,1987_pinduse,1988_pinduse,1989_pinduse,1990_pinduse,1991_pinduse,1992_pinduse,1993_pinduse,1994_pinduse,1995_pinduse,1996_pinduse,1997_pinduse,1998_pinduse,1999_pinduse,2000_pinduse,2001_pinduse,2002_pinduse,2003_pinduse,2004_pinduse,2005_pinduse,2006_pinduse,2007_pinduse,2008_pinduse,2009_pinduse,2010_pinduse,2011_pinduse,2012_pinduse,2013_pinduse,2014_pinduse,2015_pinduse,2016_pinduse,1980_pindww,1981_pindww,1982_pindww,1983_pindww,1984_pindww,1985_pindww,1986_pindww,1987_pindww,1988_pindww,1989_pindww,1990_pindww,1991_pindww,1992_pindww,1993_pindww,1994_pindww,1995_pindww,1996_pindww,1997_pindww,1998_pindww,1999_pindww,2000_pindww,2001_pindww,2002_pindww,2003_pindww,2004_pindww,2005_pindww,2006_pindww,2007_pindww,2008_pindww,2009_pindww,2010_pindww,2011_pindww,2012_pindww,2013_pindww,2014_pindww,2015_pindww,2016_pindww,1980_plivuse,1981_plivuse,1982_plivuse,1983_plivuse,1984_plivuse,1985_plivuse,1986_plivuse,1987_plivuse,1988_plivuse,1989_plivuse,1990_plivuse,1991_plivuse,1992_plivuse,1993_plivuse,1994_plivuse,1995_plivuse,1996_plivuse,1997_plivuse,1998_plivuse,1999_plivuse,2000_plivuse,2001_plivuse,2002_plivuse,2003_plivuse,2004_plivuse,2005_plivuse,2006_plivuse,2007_plivuse,2008_plivuse,2009_plivuse,2010_plivuse,2011_plivuse,2012_plivuse,2013_plivuse,2014_plivuse,2015_plivuse,2016_plivuse
0,89.75,-179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,89.25,-179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,88.75,-179.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

Taking a look at valid rows as well:

In [68]:
df_yearly_concat[df_yearly_concat['1980_pdomuse'].notnull()].head(10)

,lat_pdomuse,lon_pdomuse,1980_pdomuse,1981_pdomuse,1982_pdomuse,1983_pdomuse,1984_pdomuse,1985_pdomuse,1986_pdomuse,1987_pdomuse,1988_pdomuse,1989_pdomuse,1990_pdomuse,1991_pdomuse,1992_pdomuse,1993_pdomuse,1994_pdomuse,1995_pdomuse,1996_pdomuse,1997_pdomuse,1998_pdomuse,1999_pdomuse,2000_pdomuse,2001_pdomuse,2002_pdomuse,2003_pdomuse,2004_pdomuse,2005_pdomuse,2006_pdomuse,2007_pdomuse,2008_pdomuse,2009_pdomuse,2010_pdomuse,2011_pdomuse,2012_pdomuse,2013_pdomuse,2014_pdomuse,2015_pdomuse,2016_pdomuse,1980_pdomww,1981_pdomww,1982_pdomww,1983_pdomww,1984_pdomww,1985_pdomww,1986_pdomww,1987_pdomww,1988_pdomww,1989_pdomww,1990_pdomww,1991_pdomww,1992_pdomww,1993_pdomww,1994_pdomww,1995_pdomww,1996_pdomww,1997_pdomww,1998_pdomww,1999_pdomww,2000_pdomww,2001_pdomww,2002_pdomww,2003_pdomww,2004_pdomww,2005_pdomww,2006_pdomww,2007_pdomww,2008_pdomww,2009_pdomww,2010_pdomww,2011_pdomww,2012_pdomww,2013_pdomww,2014_pdomww,2015_pdomww,2016_pdomww,1980_pinduse,1981_pinduse,1982_pinduse,1983_pinduse,1984_pinduse,1985_pinduse,1986_pinduse,1987_pinduse,1988_pinduse,1989_pinduse,1990_pinduse,1991_pinduse,1992_pinduse,1993_pinduse,1994_pinduse,1995_pinduse,1996_pinduse,1997_pinduse,1998_pinduse,1999_pinduse,2000_pinduse,2001_pinduse,2002_pinduse,2003_pinduse,2004_pinduse,2005_pinduse,2006_pinduse,2007_pinduse,2008_pinduse,2009_pinduse,2010_pinduse,2011_pinduse,2012_pinduse,2013_pinduse,2014_pinduse,2015_pinduse,2016_pinduse,1980_pindww,1981_pindww,1982_pindww,1983_pindww,1984_pindww,1985_pindww,1986_pindww,1987_pindww,1988_pindww,1989_pindww,1990_pindww,1991_pindww,1992_pindww,1993_pindww,1994_pindww,1995_pindww,1996_pindww,1997_pindww,1998_pindww,1999_pindww,2000_pindww,2001_pindww,2002_pindww,2003_pindww,2004_pindww,2005_pindww,2006_pindww,2007_pindww,2008_pindww,2009_pindww,2010_pindww,2011_pindww,2012_pindww,2013_pindww,2014_pindww,2015_pindww,2016_pindww,1980_plivuse,1981_plivuse,1982_plivuse,1983_plivuse,1984_plivuse,1985_plivuse,1986_plivuse,1987_plivuse,1988_plivuse,1989_plivuse,1990_plivuse,1991_plivuse,1992_plivuse,1993_plivuse,1994_plivuse,1995_plivuse,1996_plivuse,1997_plivuse,1998_plivuse,1999_plivuse,2000_plivuse,2001_plivuse,2002_plivuse,2003_plivuse,2004_plivuse,2005_plivuse,2006_plivuse,2007_plivuse,2008_plivuse,2009_plivuse,2010_plivuse,2011_plivuse,2012_plivuse,2013_plivuse,2014_plivuse,2015_plivuse,2016_plivuse
37,71.25,-179.75,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000